In [2]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree as tr
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance
from xgboost import XGBRegressor

In [3]:
# load data
data_path = "../data/cesd_total.csv"
data = pd.read_csv(data_path)

In [4]:
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.15, 
                                                    random_state=42)

In [6]:
# let's further split the training set into a training and a validation set (15%)
X_train, X_val, y_train, y_val = train_test_split(X_train, 
                                                  y_train,
                                                  test_size=X_test.shape[0] / X_train.shape[0],  
                                                  random_state=42)

In [7]:
def identify_variable_types(data):
    continuous_vars = []
    dummy_vars = []

    num_columns = data.shape[1]
    
    for i in range(num_columns):
        unique_values = np.unique(data[:, i])
        if len(unique_values) == 2 and np.array_equal(unique_values, [0, 1]):
            dummy_vars.append(i)
        else:
            continuous_vars.append(i)
    
    return continuous_vars, dummy_vars

In [8]:
continuous_cols_train, dummy_cols_train = identify_variable_types(X_train)

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_cols_train),
        ('dummy', 'passthrough', dummy_cols_train)  # Leave dummy variables unchanged or use StandardScaler() if needed
    ]
)

# Fit and transform the data
X_train = preprocessor.fit_transform(X_train)

continuous_cols_val, dummy_cols_val = identify_variable_types(X_val)

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_cols_val),
        ('dummy', 'passthrough', dummy_cols_val)  # Leave dummy variables unchanged or use StandardScaler() if needed
    ]
)

# Fit and transform the data
X_val = preprocessor.fit_transform(X_val)

continuous_cols_test, dummy_cols_test = identify_variable_types(X_test)

# Define the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), continuous_cols_test),
        ('dummy', 'passthrough', dummy_cols_test)  # Leave dummy variables unchanged or use StandardScaler() if needed
    ]
)

# Fit and transform the data
X_test = preprocessor.fit_transform(X_test)

In [9]:
model_performances = []

In [10]:
def run_on_splits(func):
    def _run_loop(*args, **kwargs):
        for x,y,nsplit in zip([X_train, X_val, X_test],
                              [y_train, y_val, y_test],
                              ['train', 'val', 'test']):
            func(*args, X=x, y=y, nsplit=nsplit, **kwargs)
    return _run_loop

In [11]:
@run_on_splits
def evaluate(model, X, y, nsplit, model_name, constant_value=None):
    ''' Evaluates the performance of a model 
    Args:
        model (sklearn.Estimator): fitted sklearn estimator
        X (np.array): predictors
        y (np.array): true outcome
        nsplit (str): name of the split
        model_name (str): string id of the model
        constant_value (int or None): relevant if the model predicts a constant
    '''
    if constant_value is not None:
        preds = np.array([constant_value] * y.shape[0])
    else:
        preds = model.predict(X)
    r2 = r2_score(y, preds)
    performance = np.sqrt(mean_squared_error(y, preds))
    model_performances.append({'model': model_name,
                         'split': nsplit,
                         'rmse': round(performance, 4),
                         'r2': round(r2, 4)})

In [12]:
model_performances = []

In [13]:
evaluate(model=None, model_name='dummy', constant_value=y_train.mean())

In [14]:
def run_dummy(X_train, X_val, X_test, y_train, y_val, y_test):
    evaluate(model=None, model_name='dummy', constant_value=y_train.mean())
    print(model_performances)

In [15]:
run_dummy(X_train, X_val, X_test, y_train, y_val, y_test)

[{'model': 'dummy', 'split': 'train', 'rmse': 11.2661, 'r2': 0.0}, {'model': 'dummy', 'split': 'val', 'rmse': 12.044, 'r2': -0.0241}, {'model': 'dummy', 'split': 'test', 'rmse': 12.5069, 'r2': -0.0051}, {'model': 'dummy', 'split': 'train', 'rmse': 11.2661, 'r2': 0.0}, {'model': 'dummy', 'split': 'val', 'rmse': 12.044, 'r2': -0.0241}, {'model': 'dummy', 'split': 'test', 'rmse': 12.5069, 'r2': -0.0051}]


In [16]:
reg = LinearRegression().fit(X_train, y_train)

In [17]:
evaluate(model=reg, model_name='linear')

In [18]:
models = {} # storing fitted models in the next chunk
models['linear-0.0'] = reg

for alpha in [0.01, 0.1, 0.2, 0.5, 1.0, 20.0, 10.0, 100.0, 1000.0]:
    for est in [Lasso, Ridge]:
        if est == Lasso:
            id = 'lasso'
        else:
            id = 'ridge'
        reg = est(alpha=alpha).fit(X_train, y_train)
        models[f'{id}-{alpha}'] = reg
        evaluate(model=reg, model_name=f'{id}-alpha-{alpha}')

In [21]:
def _compute_bias(pred, true):
    ''' Function to compute bias. 
        Note that here we compute the average squared bias of the model
        over all data points to get a sense for its tendency to make 
        systematically "off-target" predictions.

    Args:
        pred (np.array): array of shape (n_samples, n_bootstrap), where n_samples is the size
            of the test set, and n_bootstraps is how many times we sample data from the training
            set and fit our model
        true (np.array): predictions of the true model. It is an array of shape (n_samples)
    '''
    mpred = pred.mean(axis=1)
    return (((true - mpred)**2)).mean()

In [22]:
def _compute_variance(pred):
    ''' Function to compute variance
    Args:
        pred (np.array): array of shape (n_samples, n_bootstrap), where n_samples is the size
            of the test set, and n_bootstraps is how many times we sample data from the training
            set and fit our model
    '''
    return pred.var(axis=1).mean()

In [23]:
max_degree = 10 # this defines the most complex model we will fit (with x, ..., x^10) as inputs
n_sim = 1000 # this defines how many times we do bootstrapping
performances = [] # here we will store performances
results = [] # here we store performances as a DF (see logic in the loop)
outputs = ['mtype', # this is the information we want to store in the DF
           'complexity', 
           'bias^2', 
           'variance', 
           'mse']

for c in range(1, max_degree):
    reg = LinearRegression() # initialize the model 
    pred_all = np.zeros(shape=(y_test.shape[0], n_sim)) # empty array, which will store our predictions across bootstraps
    for sim in range(n_sim): # bootstrap n_sim times
        x, _, y, _ = train_test_split(X_train, y_train, train_size=.5) # sample half of the data
        transformer = PolynomialFeatures(degree=c)
        x_fit = transformer.fit_transform(x[:,1].reshape(-1,1)) # take x, and add all polynomials all the way to c (current complexity)
        x_test = transformer.fit_transform(X_test[:,1].reshape(-1,1)) # do the same with the test set
        reg.fit(x_fit,y) # fit the model
        preds = reg.predict(x_test) # predict the outcomes
        pred_all[:,sim] = preds # store that in the big result matrix

    # once we are done bootstrapping, we have predictions for all models
    mse = np.mean([np.sqrt(mean_squared_error(y_test, 
                                              pred_all[:,sim])) # compute the average MSE of the models
                    for sim in range(n_sim)])
    bias = _compute_bias(pred_all, y_test) # get bias
    variance = _compute_variance(pred_all) # get variance
    info = ('linear', x_fit.shape[1]-1, bias, variance, mse)
    performances.append(info) # append model info to performances

# Getting the results into a dataframe 
result = pd.DataFrame(performances, columns=outputs)
for c in ['bias^2', 'variance', 'mse']:
    result[f'{c}-scaled'] = MinMaxScaler().fit_transform(result[c].values.reshape(-1,1)) # rescale, so bias and variance are on the same scale
results.append(result)

In [24]:
results

[    mtype  complexity      bias^2     variance        mse  bias^2-scaled  \
 0  linear           1  159.392657     0.355622  12.638962       0.000000   
 1  linear           2  159.544480     0.475238  12.649676       0.000842   
 2  linear           3  159.761346     0.695780  12.666909       0.002044   
 3  linear           4  160.288656     1.092431  12.703008       0.004968   
 4  linear           5  160.013227     1.941988  12.723786       0.003441   
 5  linear           6  160.332740     6.413099  12.872980       0.005212   
 6  linear           7  160.160580    36.989057  13.362098       0.004258   
 7  linear           8  170.943303   423.518532  15.186416       0.064041   
 8  linear           9  339.754791  7660.842021  27.102557       1.000000   
 
    variance-scaled  mse-scaled  
 0         0.000000    0.000000  
 1         0.000016    0.000741  
 2         0.000044    0.001932  
 3         0.000096    0.004428  
 4         0.000207    0.005865  
 5         0.000791    0

In [25]:
model_performances

[{'model': 'dummy', 'split': 'train', 'rmse': 11.2661, 'r2': 0.0},
 {'model': 'dummy', 'split': 'val', 'rmse': 12.044, 'r2': -0.0241},
 {'model': 'dummy', 'split': 'test', 'rmse': 12.5069, 'r2': -0.0051},
 {'model': 'dummy', 'split': 'train', 'rmse': 11.2661, 'r2': 0.0},
 {'model': 'dummy', 'split': 'val', 'rmse': 12.044, 'r2': -0.0241},
 {'model': 'dummy', 'split': 'test', 'rmse': 12.5069, 'r2': -0.0051},
 {'model': 'linear', 'split': 'train', 'rmse': 6.9475, 'r2': 0.6197},
 {'model': 'linear',
  'split': 'val',
  'rmse': 55978618892232.44,
  'r2': -2.2122366196762484e+25},
 {'model': 'linear',
  'split': 'test',
  'rmse': 45297602434711.734,
  'r2': -1.3183983522923074e+25},
 {'model': 'lasso-alpha-0.01', 'split': 'train', 'rmse': 7.1045, 'r2': 0.6023},
 {'model': 'lasso-alpha-0.01', 'split': 'val', 'rmse': 12.3959, 'r2': -0.0848},
 {'model': 'lasso-alpha-0.01', 'split': 'test', 'rmse': 11.7864, 'r2': 0.1074},
 {'model': 'ridge-alpha-0.01', 'split': 'train', 'rmse': 6.9584, 'r2': 0.6